In [1]:
# Preparations
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import seaborn as sns
from IPython.display import Latex
import warnings
from PrettyTable import PrettyTable
from functools import partial
from PrettyFigure import PrettyFigure
warnings.filterwarnings("ignore", module="matplotlib")
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.rcParams['savefig.dpi'] = 75

# plt.rcParams['figure.autolayout'] = False
# plt.rcParams['figure.figsize'] = 10, 6
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14

plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.unicode'] = True
plt.rcParams['font.family'] = "STIX"
plt.rcParams['text.latex.preamble'] = "\\usepackage{subdepth}, \\usepackage{type1cm}"

results = {}

sns.set(color_codes=True)

def average(data):
    return 1 / len(data) * sum(data)

def error(data, average_of_data):
    s = sum([(x - average_of_data)**2 for x in data])
    return math.sqrt(s / (len(data) * (len(data) - 1)))

def std_deviation(error_of_average, length_of_dataset):
    return error_of_average * math.sqrt(length_of_dataset)

def average_with_weights(data, weights):
    d = data
    w = weights
    return (d * w**-2).sum() / (w**-2).sum()

def error_with_weights(weights):
    w = weights
    return 1 / math.sqrt((w**-2).sum())

def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    return (d * w**-2).sum() / (w**-2).sum()

def werr(group, weight_name):
    return 1 / math.sqrt((group[weight_name]**-2).sum())

In [2]:
# Constants
c0 = 299792458 # [m / s]
f1 = 1 # [m]
f2 = 4.994 # [m]
s1 = 0.110 # [m]
s2 = 4.873 # [m]
AE = 0.4 # [g / m^3]
fmax = 1 # [g / m^3]
phi = f / fmax
T = 23 # [K]
ps = 0.03 # [Pa]

nn = 0
pn = 0 # [Pa]
Tn = 0 # [K]
beta = 4.292e-8 # [1 / mbar]
gamma = 3.43e-2 # [(nm)^2 / mbar]
lambda0 = 632.8 #[nm]
pw = f

#n = (nn - 1) * p * Tn / (pn * T) - (beta - gamma / lambda0**2) * pw + 1

#ctheoretisch = c0 / n # [m / s]

In [3]:
#x = (1 / c) * 2 * omega * (s2 + fs) * (2 * f1) + x0

# Read Data
dfb = pd.read_csv('data/measurements.csv')
ax = None
dfa = dfb.loc[dfb['observer'] == 'Almar']
dfn = dfb.loc[dfb['observer'] == 'Noah']

slopea, intercepta, sem, r, p = stats.linregress(dfa['f'], dfa['x'])
slopen, interceptn, sem, r, p = stats.linregress(dfn['f'], dfn['x'])

ax = dfa.plot(kind='scatter', x='f', y='x', label='gemessene Distanz [Almar]')
plt.plot(dfa['f'], dfa['f'] * slopea + intercepta, label='linearer Fit der Distanz [Almar]', axes=ax)
plt.plot(dfn['f'], dfn['f'] * slopen + interceptn, label='linearer Fit der Distanz [Noah]', axes=ax, color='red')
plt.scatter(dfn['f'], dfn['x'], label='gemessene Distanz [Noah]', axes=ax, color='red')
plt.xlabel('Frequenz [Hz]')
plt.ylabel('Strecke [m]')
plt.legend(bbox_to_anchor=(0.02, 0.98), loc=2, borderaxespad=0.2)

plt.close()
figure = PrettyFigure(
    ax.figure,
    label='fig:lichtgeschwindigkeiten_distanz_Almar',
    caption='Gemessene Distanzen. Dazu einen linearen Fit um die Mittlere Distanz zu bestimmen.'
)
figure.show()

c = (1 / ((slopen + slopea) / 2 / 1e4)) * 2 * 2 * math.pi * (s2 + f2) * (2 * f1)
print(c)

304783053.829


In [ ]:
V = [15, 12.5, 10, 7.5, 5, 2.5]
rpm = [863, 718, 566, 411, 269, 106]

plt.plot(V, rpm)

# Arbeitsgrundlagen

## Lichtgeschwindigkeit in der Theorie

Die Lichtgeschwindigkeit im Vakuum wurde 1983 durch die Internationale Konferenz für Mass und Gewicht (ICPM) auf exakt

<center>
$c_0 = 299'792'458 \frac{m}{s}$
</center>
    
festgelegt.

Im Medium kann die Lichtgeschwindigkeit über den Brechungsindex $n$ bestimmt werden mit

\begin{equation}
c = \frac{c_0}{n}
\label{eq:c}
\end{equation}

Der Brechungsindex wiederum kann durch

\begin{equation}
n = (n_n - 1) \cdot \frac{pT_n}{p_nT} - \Big(\beta-\frac{\gamma}{\lambda_0^2}\Big)\cdot p_w -1
\label{eq:n}
\end{equation}

bestimmt werden. Hierbei gilt:

<center>
n_n = Normbrechungsindex für trockene Luft  
$\beta = 4.292\cdot10^{-8}mbar^{-1}$  
$\gamma = 3.43\cdot10^{-2}(nm)^2mbar^{-1}$  
$\lambda_0 \approx \lambda = 632.8 nm$  
$p_w = AE \cdot p_s = 0.4 \cdot 0.03 \cdot 10^3 mbar$
</center>

Somit errechnet sich die theoretische Lichtgeschwindigkeit im Zimmer als $c = 0\frac{m}{s}$.

## Messung der Lichtgeschwindigkeit nach Michelson

Beim Versuch von Michelson geht es drum den Lichtstrahl eines Lasers über verschiedene Spiegel, Linsen und Blenden zu schicken. Etwa in der mitte der totalen Distanz trifft der Strahl auf einen rotierenden Laser. Da das Licht auch nur endlich schnell ist, trifft das Licht auf dem Rückweg nicht an der selben Stelle auf dem rotierenden Spiegel auf wie auf dem Hinweg.
Somit kann über die zurückgelegte Distanz und die Verschiebung des Aufpralls auf dem Spiegel, sowie der bekannten Drehzahl die Lichtgeschwindigkeit errechnet werden.

![Versuchsaufbau nach Michelson. Ennommen aus der Aufgabenstellung](images/michelson.png)

Der Versuchsaufbau kann äquialent wie folgt dargestellt werden

![Äuquvaente Linsenkonfiguration nach Michelson. Ennommen aus der Aufgabenstellung](images/aequiv.png)

Der rotierende Spiegel wird zwischen dem ersten und dem zweiten Reflektieren des Strahls um den Winkel $\delta$ gedreht

\begin{equation}
\delta = \omega\Delta t = \omega \frac{2(S_2+D_2)}{c}
\label{eq:delta}
\end{equation}
